In [1]:
from src.controller.HwpController import HwpController

import re, uuid
import numpy as np

In [2]:
controller = HwpController()
path = '/Users/dobby/Desktop/Workspace/3월 계측보고서(김해상동물류)_간단이.hwp'

controller.get_table_list(path, '일자별 계측 현황')
table_list = controller.get_list()

table_list

[[{'row': '0', 'col': '0', 'rowspan': '1', 'text': '3월4일'},
  {'row': '1', 'col': '0', 'rowspan': '2', 'text': '구분'},
  {'row': '1', 'col': '7', 'rowspan': '2', 'text': '허용기준'},
  {'row': '1', 'col': '8', 'rowspan': '2', 'text': '비고'},
  {'row': '2', 'col': '1', 'rowspan': '1', 'text': '최저치'},
  {'row': '2', 'col': '2', 'rowspan': '1', 'text': '최고치'},
  {'row': '2', 'col': '3', 'rowspan': '1', 'text': '최저치'},
  {'row': '2', 'col': '4', 'rowspan': '1', 'text': '최고치'},
  {'row': '2', 'col': '5', 'rowspan': '1', 'text': '최저치'},
  {'row': '2', 'col': '6', 'rowspan': '1', 'text': '최고치'},
  {'row': '3', 'col': '0', 'rowspan': '1', 'text': '대감터널'},
  {'row': '3', 'col': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '2', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '3', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '4', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '5', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '6', 'rowspan': '1', 'text': 'N/T

In [6]:
def delete_non_target_data(table_data):
    """
    한글 표에서 원하지 않은 부분까지 나온 데이터를 정리하여 리스트로 반환합니다.
    target_data_text에 표에 반복적으로 들어가는 텍스트를 입력하여 필요없는 데이터를 삭제합니다.
    """
    target_data_text = ['진동레벨', '진동속도', '소음레벨']

    target_data = [
        sublist for sublist in table_data
        if any(
            entry['row'] in ['0', '1'] and any(keyword in entry['text'] for keyword in target_data_text)
            for entry in sublist
        )
    ]

    return target_data

In [7]:
table_list = delete_non_target_data(table_list)

table_list

[]

In [6]:
def extract_columns(table_list):
    """
    한글 표에서 공통적인 컬럼 부분을 추출합니다.
    컬럼은 대부분 표의 시작 부분에 작성되기 때문에 row 값은 0 혹은 1에 위치하게 됩니다.
    이후 컬럼 값들이 딕셔너리로 저장되어 있기 때문에 중복을 제거 후 리스트로 반환합니다.
    """
    columns = [[item['text'] for item in items if item['row'] == '0' or item['row'] == '1'] for items in table_list ]
    columns = set(tuple(items) for items in columns)
    
    return [item for items in columns for item in items]

In [10]:
table_list

[]